# Çok Değişkenli Aykırı Gözlem Analizi

## Local Outlier Factor

Gözlemleri bulundukları konumda yoğunluk tabanlı skorlayarak buna göre aykırı değer olabilecek değerleri tanımlayabilmemize imkan sağlıyor.

Bir noktanın local yoğunluğu bu noktanın komşuları ile karşılaştırılıyor. Eğer bir nokta komşularınının yoğunluğundan anlamlı şekilde düşük ise bu nokta komşularından daha seyrek bir bölgede bulunuyordur yorumu yapılabiliyor. Dolayısıyla burada bir komşuluk yapısı söz konusu. Bir değerin çevresi yoğun değil ise demek ki bu değer aykırı değerdir şeklinde değerlendiriliyor.

tek değişkenlide boxplot en sık kullanılan yöntemdir. sektör bilgisiyle elbette. çok değişkenlide ise, lof(local outlier factor) en sık kullanılanlar arasındadır.

neden veri setine çok değişkenli olarak bakmalıyız, elimizde 2 değişken olsun. Birisi yaş değişkeni olsun, birisi de evlilik sayısı değişkeni olsun. yaş değişkenini tek başına göz önünde bulundurulalım. örneğin yaş[17,18,70,80] bunlar aykırı değerler değildir. Bir yaş için olabilecek değerlerdir. evlilik sayısı[1,2,3,1] yine bu değerler, gözlemlenebilecek değerlerdir. aykırı değerler değildir. Şimdi iki değişkeni ayrı ayrı göz önünde bulundurduğumuzda, iki değişken için de aykırı bir durum söz konusu değil. Ancak, aynı anda göz önünde bulundurursak: 
 yaş | evlilik sayısı
 17         3
 18         2
 70         3
 80         1
 Burada yaşı 17 olup evlilik sayısı 3 olan bir değer var. bu genel bir durum değil yani gözlemlenebilecek bir durum değildir. eğer ki yaşanmışsa da olmuşsa da bu zaten aykırı bir durumdur ve bunu yakalamış olduk. evet olabilir ama çok uç bir senaryodur.
 
değişkenler tek başınayken aykırı gözlem şeklinde algılanamayacak şeklinde olan durumlar, değişkenler çok değişkenli olacak şekilde eş zamanlıca göz önünde bulundurulduğunda, ortaya hesapta olmayan aykırı gözlemler ortaya çıkabilmektedir.


In [3]:
import seaborn as sns
df = sns.load_dataset("diamonds")
df = df.select_dtypes(include = ["float64", "int64"])
df = df.dropna()
df.head()

,carat,depth,table,price,x,y,z
0,0.23,61.5,55.0,326,3.95,3.98,2.43
1,0.21,59.8,61.0,326,3.89,3.84,2.31
2,0.23,56.9,65.0,327,4.05,4.07,2.31
3,0.29,62.4,58.0,334,4.20,4.23,2.63
4,0.31,63.3,58.0,335,4.34,4.35,2.75


In [4]:
import numpy as np
from sklearn.neighbors import LocalOutlierFactor #şimdi lof kullanarak bir skorlama işlemi gerçekleştiricez

In [5]:
clf = LocalOutlierFactor(n_neighbors = 20, contamination = 0.1) #n_neighbours= komşuluk sayısını ifade ediyor,
#contamination ise yoğunluğu ifade ediyor.

In [6]:
clf.fit_predict(df) #clf nesnemiz loc.outliersın biçimsel özelliklerini barındırıyor. Şimdi fit etme işlemi gerçekleştiricez
#artık elimizde her bir gözlem birimi için elde edilmiş skorlar olacak.

array([-1, -1, -1, ...,  1,  1,  1])

In [7]:
df_scores = clf.negative_outlier_factor_ 

In [8]:
df_scores[0:10] #elimizde, tüm gözlemler için bir skor değeri, bir eşik değeri olarak kabul edilebilecek değerler var.
#bunlar her bir gözlem değerinin yoğunluk skoru.

array([-1.58352526, -1.59732899, -1.62278873, -1.33002541, -1.30712521,
       -1.28408436, -1.28428162, -1.26458706, -1.28422952, -1.27351342])

In [8]:
np.sort(df_scores[0:10]) #sort ile sıralıyoruz, incelediğimizde çok farklılaşmadığı gözlemleniyor.

array([-1.62278873, -1.59732899, -1.58352526, -1.33002541, -1.30712521,
       -1.28428162, -1.28422952, -1.28408436, -1.27351342, -1.26458706])

In [9]:
np.sort(df_scores[0:20]) #burada da hep yakın değerler. Burada, belirleyecek olduğumuz bir eşik değere kadar bakmayı sürdürmemiz gerekiyor
#mesela bir değer belirlediğimizi varsayalım. Kafadan bir değer ama kurala göre gibiymiş gibi. 10. değeri eşik değer skoru olarak
#belirleyelim. bu skordan aşağısında kalan değerleri aykırı değer olarak belirleyelim.

array([-1.62278873, -1.59732899, -1.58352526, -1.33002541, -1.30712521,
       -1.28428162, -1.28422952, -1.28408436, -1.27351342, -1.26458706,
       -1.23611906, -1.21093135, -1.20830179, -1.17637026, -1.14305176,
       -1.13307628, -1.13060731, -1.11364018, -1.08139269, -1.08093519])

In [10]:
np.sort(df_scores)[13] #farkettik ki yukarıda yanlış yapıldı. önce sort edip sonra ilk 20 değere bakmalıydık.

-3.823787967755565

In [9]:
np.sort(df_scores)[0:20]

array([-8.60430658, -8.20889984, -5.86084355, -4.98415175, -4.81502092,
       -4.81502092, -4.61522833, -4.37081214, -4.29842288, -4.10492387,
       -4.0566648 , -4.01831733, -3.94882806, -3.82378797, -3.80135297,
       -3.75680919, -3.65947378, -3.59249261, -3.55564138, -3.47157375])

In [10]:
np.sort(df_scores)[13] #şimdi bunu eşik değer kabul edip buna göre işlemlerimize devam edelim.

-3.823787967755565

In [11]:
esik_deger = np.sort(df_scores)[13]

In [12]:
aykiri_tf = df_scores > esik_deger #diyoruz ki, score arrayini karşılaştıralım eşik değerden yukarıda olanları aykırıtf olarak kaydedelim.

In [13]:
aykiri_tf

array([ True,  True,  True, ...,  True,  True,  True])

### Silme Yöntemi

In [14]:
yeni_df = df[df_scores > esik_deger] #ne yaptık, eşik değerin üzerindeli bütün değerleri aldık yani aykırı olmayan değerlere eriştik
# bu aykırı değerleri silmek anlamına geliyor. bu değerler aykırı olmayan değerler.

In [15]:
yeni_df

,carat,depth,table,price,x,y,z
0,0.23,61.5,55.0,326,3.95,3.98,2.43
1,0.21,59.8,61.0,326,3.89,3.84,2.31
2,0.23,56.9,65.0,327,4.05,4.07,2.31
3,0.29,62.4,58.0,334,4.20,4.23,2.63
4,0.31,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...
53935,0.72,60.8,57.0,2757,5.75,5.76,3.50
53936,0.72,63.1,55.0,2757,5.69,5.75,3.61
53937,0.70,62.8,60.0,2757,5.66,5.68,3.56
53938,0.86,61.0,58.0,2757,6.15,6.12,3.74


In [16]:
df[df_scores<esik_deger] #bunlar ise aykırı değerlerdir.

,carat,depth,table,price,x,y,z
6341,1.00,44.0,53.0,4032,6.31,6.24,4.12
10377,1.09,43.0,54.0,4778,6.53,6.55,4.12
24067,2.00,58.9,57.0,12210,8.09,58.90,8.06
35633,0.29,62.8,44.0,474,4.20,4.24,2.65
36503,0.30,51.0,67.0,945,4.67,4.62,2.37
38840,0.73,70.8,55.0,1049,5.51,5.34,3.84
41918,1.03,78.2,54.0,1262,5.72,5.59,4.42
45688,0.70,71.6,55.0,1696,5.47,5.28,3.85
48410,0.51,61.8,54.7,1970,5.12,5.15,31.80
49189,0.51,61.8,55.0,2075,5.15,31.80,5.12


# Baskılama

In [1]:
#ortalama için burda her bir değişken için aykırılıklara gidip ortalamayı atamak gerekiyor.
#ncak burada baskılamada, çok değişkenlide en mantıklı yaklaşım.

In [17]:
df[df_scores == esik_deger] #Şimdi bu belirlediğimiz eşik değer, bunu nasıl bulduk? 
#lof ile yoğunluk tabanlı bir skor oluşturmuştuk. bunun içerisinden 13. değeri eşik değer olarak belirledik.
#işte bu değer belirlediğimiz 13. değer. bu eşik değerin belirlemiş olduğumuz skora karşılık gelen nümerik
#gözlem birimi de bu aşağıdaki. şimdi şunu yapacağız: bu gözlem biriminin değerlerini, 
#aykırı gözlemlere atayacağız

,carat,depth,table,price,x,y,z
31230,0.45,68.6,57.0,756,4.73,4.5,3.19


In [18]:
baski_deger = df[df_scores == esik_deger]#aykırıları da yukarıda belirtmiştik tekrar görelim ve düzenleyelim

In [22]:
aykirilar = df[~aykiri_tf]

In [23]:
#açıklama: elimizdeki mevcut aykırıların yerine eşik değerdeki değerleri atayacak olduğumuzdan dolayı,
#bazı index problemleri ortaya çıkıyo. bunları gidermek için birkaç işlem yapacağız.
#önce aykırılar dataframini indexsiz bir arraye çeviricez. sonra, baskı gözlemini de arrayleştirip,
# atama işlemini gerçekleştiricez. son olarak da, aykırı değerleri, baskı değeri ile değiştirmiş olucaz.

In [24]:
res = aykirilar.to_records(index = False)

In [26]:
aykirilar

,carat,depth,table,price,x,y,z
6341,1.00,44.0,53.0,4032,6.31,6.24,4.12
10377,1.09,43.0,54.0,4778,6.53,6.55,4.12
24067,2.00,58.9,57.0,12210,8.09,58.90,8.06
31230,0.45,68.6,57.0,756,4.73,4.50,3.19
35633,0.29,62.8,44.0,474,4.20,4.24,2.65
36503,0.30,51.0,67.0,945,4.67,4.62,2.37
38840,0.73,70.8,55.0,1049,5.51,5.34,3.84
41918,1.03,78.2,54.0,1262,5.72,5.59,4.42
45688,0.70,71.6,55.0,1696,5.47,5.28,3.85
48410,0.51,61.8,54.7,1970,5.12,5.15,31.80


In [25]:
res #ne yaptık? aykirilar dataframeini, numpy arrayine çevirdi. indexlerinden kurtulduk.

rec.array([(1.  , 44. , 53. ,  4032, 6.31,  6.24,  4.12),
           (1.09, 43. , 54. ,  4778, 6.53,  6.55,  4.12),
           (2.  , 58.9, 57. , 12210, 8.09, 58.9 ,  8.06),
           (0.45, 68.6, 57. ,   756, 4.73,  4.5 ,  3.19),
           (0.29, 62.8, 44. ,   474, 4.2 ,  4.24,  2.65),
           (0.3 , 51. , 67. ,   945, 4.67,  4.62,  2.37),
           (0.73, 70.8, 55. ,  1049, 5.51,  5.34,  3.84),
           (1.03, 78.2, 54. ,  1262, 5.72,  5.59,  4.42),
           (0.7 , 71.6, 55. ,  1696, 5.47,  5.28,  3.85),
           (0.51, 61.8, 54.7,  1970, 5.12,  5.15, 31.8 ),
           (0.51, 61.8, 55. ,  2075, 5.15, 31.8 ,  5.12),
           (0.81, 68.8, 79. ,  2301, 5.26,  5.2 ,  3.58),
           (0.5 , 79. , 73. ,  2579, 5.21,  5.18,  4.09),
           (0.5 , 79. , 73. ,  2579, 5.21,  5.18,  4.09)],
          dtype=[('carat', '<f8'), ('depth', '<f8'), ('table', '<f8'), ('price', '<i8'), ('x', '<f8'), ('y', '<f8'), ('z', '<f8')])

In [28]:
res[:] = baski_deger.to_records(index = False) #bu res in içerisindeki tüm değerleri ([:]) şöyle oluştur:
#baski degeri alıyoruz, bunu da arraye çevir diyoruz,yani az önce oluşturduğumuz res lerin yerine
# baskı değeri yerleştirecek.

In [29]:
res # bütün aykırıların yerine eşik değerimiz geldi. yani baskıladık.
#burada şöyle ir sorun var, bu ayrı bir array, aykırılar ayrı bir array, 
#ama ana dfimiz içerinde bu yapı böyle değil. bakalım: 

rec.array([(0.45, 68.6, 57., 756, 4.73, 4.5, 3.19),
           (0.45, 68.6, 57., 756, 4.73, 4.5, 3.19),
           (0.45, 68.6, 57., 756, 4.73, 4.5, 3.19),
           (0.45, 68.6, 57., 756, 4.73, 4.5, 3.19),
           (0.45, 68.6, 57., 756, 4.73, 4.5, 3.19),
           (0.45, 68.6, 57., 756, 4.73, 4.5, 3.19),
           (0.45, 68.6, 57., 756, 4.73, 4.5, 3.19),
           (0.45, 68.6, 57., 756, 4.73, 4.5, 3.19),
           (0.45, 68.6, 57., 756, 4.73, 4.5, 3.19),
           (0.45, 68.6, 57., 756, 4.73, 4.5, 3.19),
           (0.45, 68.6, 57., 756, 4.73, 4.5, 3.19),
           (0.45, 68.6, 57., 756, 4.73, 4.5, 3.19),
           (0.45, 68.6, 57., 756, 4.73, 4.5, 3.19),
           (0.45, 68.6, 57., 756, 4.73, 4.5, 3.19)],
          dtype=[('carat', '<f8'), ('depth', '<f8'), ('table', '<f8'), ('price', '<i8'), ('x', '<f8'), ('y', '<f8'), ('z', '<f8')])

In [30]:
df[~aykiri_tf] #bakın aykırılar değişmemiş. yani yukarıdaki arrayi, bu dataframein içine yerleştirmek gerek.
#bunun için de önce bu arrayi dataframe e çevirmemiz, daha sonra bu değerlerin basitçe atamamız gerek.

,carat,depth,table,price,x,y,z
6341,1.00,44.0,53.0,4032,6.31,6.24,4.12
10377,1.09,43.0,54.0,4778,6.53,6.55,4.12
24067,2.00,58.9,57.0,12210,8.09,58.90,8.06
31230,0.45,68.6,57.0,756,4.73,4.50,3.19
35633,0.29,62.8,44.0,474,4.20,4.24,2.65
36503,0.30,51.0,67.0,945,4.67,4.62,2.37
38840,0.73,70.8,55.0,1049,5.51,5.34,3.84
41918,1.03,78.2,54.0,1262,5.72,5.59,4.42
45688,0.70,71.6,55.0,1696,5.47,5.28,3.85
48410,0.51,61.8,54.7,1970,5.12,5.15,31.80


In [31]:
import pandas as pd
df[~aykiri_tf] = pd.DataFrame(res, index = df[~aykiri_tf].index)

In [32]:
df[~aykiri_tf]

,carat,depth,table,price,x,y,z
6341,0.45,68.6,57.0,756,4.73,4.5,3.19
10377,0.45,68.6,57.0,756,4.73,4.5,3.19
24067,0.45,68.6,57.0,756,4.73,4.5,3.19
31230,0.45,68.6,57.0,756,4.73,4.5,3.19
35633,0.45,68.6,57.0,756,4.73,4.5,3.19
36503,0.45,68.6,57.0,756,4.73,4.5,3.19
38840,0.45,68.6,57.0,756,4.73,4.5,3.19
41918,0.45,68.6,57.0,756,4.73,4.5,3.19
45688,0.45,68.6,57.0,756,4.73,4.5,3.19
48410,0.45,68.6,57.0,756,4.73,4.5,3.19


In [ ]:
#e yaptık: orjinal df içerisindeki aykırıları baştan tanımlıyoruz(df[~aykiri_tf]= ile). nasıl? pd.dataframe
#yani bir datframe oluşturulacak, res değerlerini df e çevir. bir de index değerlerini ben belirleyeyim.
# onları da şuradan al: df[~aykiri_tf] buradaki df in index değerleri olarak al. yani buradaki
# index değerleriyle beraber resleri yerleştir. zira reslerin indexleri yok.
# neyin indexini vericez? aykırı değerlerin indexini veriyoruz.